<a href="https://colab.research.google.com/github/demonlexe/cs4375-group17/blob/main/validation_blip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>